# Final Project

### Students: Nafis Banirazi & Jan Carbonell

### Lab Objective:
The Objective of this project are the following: 


In [1]:
#intial set of imports
import nltk
from nltk.corpus import wordnet, stopwords
from nltk.metrics import jaccard_distance, edit_distance
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.wsd import lesk
import pandas as pd
import string
import regex

#variable initialization and instantiation
tests = []
tests_lem = []
gold_std_train = []
gold_std_test = []
lem1 = []
lem2 = []


## 1. Read all pairs of sentences of the train and test set
We proceed to open and read the input file

In [2]:
def text_to_sentences(filename):
    sentence_pair_array = []
    for line in open(filename, encoding="UTF8").readlines():
        sentence_pair_array.append([s.strip() for s in line.split("\t")])
    return sentence_pair_array

# TRIAL TESTING
trial_input = text_to_sentences('./00_data/trial/STS.input_fixed.txt')
trial_classes = open('./00_data/trial/STS_fixed.gs.txt', encoding="utf-8-sig").readlines()

# TRAINING PHASE
train_input = text_to_sentences('./00_data/train/STS.input.MSRpar_vid_SMT.txt')
train_classes = open('./00_data/train/STS.gs.MSRpar_vid_SMT.txt', encoding="utf-8-sig").readlines()

# TESTING PHASE
test_input = text_to_sentences('./00_data/test-gold/STS.input.ALL.txt')
test_classes = open('./00_data/test-gold/STS.gs.ALL.txt', encoding="UTF8").readlines()

We implement the lematizer. 

## 2. Noisy entities removal functions
### Stopwords, URL's, Punctuations

In [3]:
# Removing the punctuation and lowering the case of a string
def preprocessing(line):
    
    line = line.lower()
    
    # Clean the text
    line = regex.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", line)
    line = regex.sub(r"what's", "what is ", line)
    line = regex.sub(r"\'s", " ", line)
    line = regex.sub(r"\'ve", " have ", line)
    line = regex.sub(r"can't", "cannot ", line)
    line = regex.sub(r"n't", " not ", line)
    line = regex.sub(r"i'm", "i am ", line)
    line = regex.sub(r"\'re", " are ", line)
    line = regex.sub(r"\'d", " would ", line)
    line = regex.sub(r"\'ll", " will ", line)
    line = regex.sub(r",", " ", line)
    line = regex.sub(r"\.", " ", line)
    line = regex.sub(r"!", " ! ", line)
    line = regex.sub(r"\/", " ", line)
    line = regex.sub(r"\^", " ^ ", line)
    line = regex.sub(r"\+", " + ", line)
    line = regex.sub(r"\-", " - ", line)
    line = regex.sub(r"\=", " = ", line)
    line = regex.sub(r"'", " ", line)
    line = regex.sub(r"(\d+)(k)", r"\g<1>000", line)
    line = regex.sub(r":", " : ", line)
    line = regex.sub(r" e g ", " eg ", line)
    line = regex.sub(r" b g ", " bg ", line)
    line = regex.sub(r" u s ", " american ", line)
    line = regex.sub(r"\0s", "0", line)
    line = regex.sub(r" 9 11 ", "911", line)
    line = regex.sub(r"e - mail", "email", line)
    line = regex.sub(r"j k", "jk", line)
    line = regex.sub(r"\s{2,}", " ", line)

    #only accept alphanum
    # [w for w in doc if w.isalpha()]  # Remove numbers and punctuation.
    #remove punctuation
    return line.translate(str.maketrans('', '', string.punctuation))

## 3. Word normalization
### Tokenization, Lemmatization

In [4]:
# extract the words from the sentence minus stopwords
def words_from_sent(sent):
    # tokenized
    tokenized = nltk.word_tokenize(sent)
    # remove stopwords and return
    return [word for word in tokenized if word not in nltk.corpus.stopwords.words('english')]

def stopwords_from_sent(sent):
    # tokenized
    tokenized = nltk.word_tokenize(sent)
    # remove stopwords and return
    return [word for word in tokenized if word in nltk.corpus.stopwords.words('english')]
    
# convert words to tokens
def pos_tag_from_words(words):
    return pos_tag(words)

# Function to get wordnet pos code
def wordnet_pos_code(tag):
    if tag.startswith('NN'):
        return wordnet.NOUN
    elif tag.startswith('VB'):
        return wordnet.VERB
    elif tag.startswith('JJ'):
        return wordnet.ADJ
    elif tag.startswith('RB'):
        return wordnet.ADV
    else:
        return None

# Tokens to lemmas using wordnet lemmatizer    
def tokens_to_lemmas(tokens):
    return list(map(pos_tag_to_lemmas, tokens))

def pos_tag_to_lemmas(token):    
    pos = wordnet_pos_code(token[1])
    if pos:
        return WordNetLemmatizer().lemmatize(token[0], pos=pos)
    return token[0]

## 4. Synset, Nammed Entity and Content Parsing


In [5]:
def lesking_sentence(pos_tagged_sentence):
    """
    Returns a sentence as the given sentece using lesker algorithms.
    The input sentence must be a pos_tagged sentence (e.g. [('The', 'DN'),
    ('sun', 'NN')]).
    """
    sentence = [i[0] for i in pos_tagged_sentence]
    result = []
    
    none_type_objects = []
    for word, tag in pos_tagged_sentence:
        # 'NoneType' object has no attribute 'name'
        try:
            result.append(lesk(sentence,word, wordnet_pos_code(tag)).name())
        except:
            result.append(word)            
    return result

## 5. Distance Metrics

## 6. Potential Networks
- MLPRegressor --> Using regressors
- Support Vector Regressor
- KNN Regressor

## 7. Evaluation

In [6]:
# evaluation imports
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr
%matplotlib inline

## Main Program

In [7]:
def lemma(sent_0, sent_1):
    print('.', end='')
    sent_0, sent_1 = preprocessing(sent_0), preprocessing(sent_1)
    #print(sent_0 + '\n' + sent_1 + '\n')
    token_0, token_1 = words_from_sent(sent_0), words_from_sent(sent_1)
    pos_0, pos_1 = pos_tag_from_words(token_0), pos_tag_from_words(token_1)
    lemmas_0, lemmas_1 = tokens_to_lemmas(pos_0), tokens_to_lemmas(pos_1)
    
    #jaccard_similarity & edit similarity
    #print ('jaccard:', float(1 - jaccard_distance(set(lemmas_0), set(lemmas_1))), 'edit_dist:', float(edit_distance(lemmas_0, lemmas_1)))
    return [float(1 - jaccard_distance(set(lemmas_0), set(lemmas_1))),float(edit_distance(lemmas_0, lemmas_1))]

In [8]:
def lesk_jaccard(sent_0, sent_1):
    print('.', end='')
    sent_0, sent_1 = preprocessing(sent_0), preprocessing(sent_1)
    #print(sent_0 + '\n' + sent_1 + '\n')
    token_0, token_1 = words_from_sent(sent_0), words_from_sent(sent_1)
    pos_0, pos_1 = pos_tag_from_words(token_0), pos_tag_from_words(token_1)
    lesk_0, lesk_1 = lesking_sentence(pos_0), lesking_sentence(pos_1)
    
    #jaccard_similarity
    #print (float(1 - jaccard_distance(set(lesk_0), set(lesk_1))), '\n')
    return float(1 - jaccard_distance(set(lesk_0), set(lesk_1)))

In [9]:
def main_feature_generation(sent_0, send_1):
    featureset = {}
    print('.', end='')
    sent_0, sent_1 = preprocessing(sent_0), preprocessing(sent_1)
    #print(sent_0 + '\n' + sent_1 + '\n')
    token_0, token_1 = words_from_sent(sent_0), words_from_sent(sent_1)
    pos_0, pos_1 = pos_tag_from_words(token_0), pos_tag_from_words(token_1)
    lemmas_0, lemmas_1 = tokens_to_lemmas(pos_0), tokens_to_lemmas(pos_1)
    
    #jaccard_similarity & edit similarity
    print ('jaccard:', float(1 - jaccard_distance(set(lemmas_0), set(lemmas_1))), 'edit_dist:', float(edit_distance(lemmas_0, lemmas_1)))
    return [float(1 - jaccard_distance(set(lemmas_0), set(lemmas_1))),float(edit_distance(lemmas_0, lemmas_1))]
    

In [10]:
#print('Training Lemmas')
#training_data_X_lemma_jaccard = [lemma(data[0], data[1])[0] for data in train_input]
#training_data_X_lemma_edit = [lemma(data[0], data[1])[1] for data in train_input]

In [11]:
print('Training data with Lemmas')
trn_X_lemma_jaccard = [lemma(data[0], data[1])[0] for data in train_input]
trn_X_lemma_edit = [lemma(data[0], data[1])[1] for data in train_input]

print('Training data with Lesk')
trn_X_lesk = [lesk_jaccard(data[0], data[1]) for data in train_input]
training_scores_y = [float(line.strip()) for line in train_classes]
print('Finished Training!\n')

print('Testing data with Lemmas')
tst_X_lemma_jaccard = [lemma(data[0], data[1])[0] for data in test_input]
tst_X_lemma_edit = [lemma(data[0], data[1])[1] for data in test_input]

print('Testing Lesk')
tst_X_lesk = [lesk_jaccard(data[0], data[1])for data in test_input]

testing_scores_y = [float(line.strip()) for line in test_classes]
print('Finished Testing!\n')

print('Results')
#maybe label encoding https://www.kaggle.com/pratsiuk/valueerror-unknown-label-type-continuous
#print(training_data_X)
#print('########################################################################\n')
#print(np.array(training_data_X).reshape(-1,1))
#print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\n')
#print(training_scores_Y)


Training data with Lemmas
..............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [ ]:
# Creating a Pandas DataFrame
df_X = pd.DataFrame(trn_X_lemma_jaccard)
df_X["trainning lemma edit"] = pd.DataFrame(trn_X_lemma_edit)
#df_X = pd.DataFrame(training_data_X_lesk)
#df_X["trainning lesk"] = pd.DataFrame(training_data_X_lesk)

df_y = pd.Series(training_scores_y)

df_X_Test = pd.DataFrame(tst_X_lemma_jaccard)
#df_X_Test = pd.DataFrame(testing_data_X_lesk)
#df_X_Test["testing lesk"] = pd.DataFrame(testing_data_X_lesk)
df_X_Test["testing lemma edit"] = pd.DataFrame(tst_X_lemma_edit)

df_X.head()

,0,trainning lemma edit
0,0.473684,7.0
1,0.500000,5.0
2,0.357143,7.0
3,0.611111,6.0
4,0.111111,14.0


In [ ]:
from sklearn.neural_network import MLPRegressor
r = MLPRegressor(max_iter=1000)
r.fit(df_X, df_y)
r.score(df_X, df_y)

# do the prediction with train --> to evaluate where the model could improve and TRAIN --> To get actual results
train_prediction = r.predict(df_X).tolist()
test_prediction = r.predict(df_X_Test).tolist()

# Evaluation of the prediction
print('Results are ready!\n')
a = pearsonr(training_scores_y, train_prediction)[0]
b = pearsonr(testing_scores_y, test_prediction)[0]
print('MLP Training Accuracy: ',round(a,3))
print('MLP Testing Accuracy: ', round(b,3))
print('MLP Drop Train-Test: ', round(a-b,3))

Results are ready!

MLP Training Accuracy:  0.758
MLP Testing Accuracy:  0.64
MLP Drop Train-Test:  0.118


In [ ]:
#using pytorch
from sklearn.neural_network import MLPRegressor
#activation tanh or logistic give better performance
r = MLPRegressor(max_iter=1000, solver='lbfgs', hidden_layer_sizes=(100,50), activation='logistic')
r.fit(df_X, df_y)
r.score(df_X, df_y)

# do the prediction with train --> to evaluate where the model could improve and TRAIN --> To get actual results
train_prediction = r.predict(df_X).tolist()
test_prediction = r.predict(df_X_Test).tolist()

# Evaluation of the prediction
print('Results are ready!\n')
d = pearsonr(training_scores_y, train_prediction)[0]
e = pearsonr(testing_scores_y, test_prediction)[0]
print('MLP Training Accuracy: ',round(a,3))
print('MLP Testing Accuracy: ', round(b,3))
print('MLP Drop Train-Test: ', round(a-b,3))

Results are ready!

MLP Training Accuracy:  0.758
MLP Testing Accuracy:  0.64
MLP Drop Train-Test:  0.118


In [ ]:
from sklearn.svm import SVR
r = SVR(gamma='scale', C=1.0, epsilon=0.2)
r.fit(df_X, df_y)
r.score(df_X, df_y)

# do the prediction with train --> to evaluate where the model could improve and TRAIN --> To get actual results
train_prediction = r.predict(df_X).tolist()
test_prediction = r.predict(df_X_Test).tolist()

# Evaluation of the prediction
print('Results are ready!\n')
f = pearsonr(training_scores_y, train_prediction)[0]
g = pearsonr(testing_scores_y, test_prediction)[0]
print('SVR Training Accuracy: ',round(f,3))
print('SVR Testing Accuracy: ', round(g,3))
print('SVR Drop Train-Test: ', round(f-g,3))

Results are ready!

SVR Training Accuracy:  0.727
SVR Testing Accuracy:  0.579
SVR Drop Train-Test:  0.148


In [ ]:
#Grid Searching SVR
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear', 'poly', 'sigmoid', 'rbf'), 'C':(0.1, 1, 10), 'epsilon':(0, 0.1, 0.2, 0.3,),  'tol':(0.0005, 0.001, 0.002, 0.003)}
r = GridSearchCV(SVR(gamma = 'scale', degree = 4), parameters)


r.fit(df_X, df_y)
print (r.best_score_)
print (r.best_params_)
r.score(df_X, df_y)

# do the prediction with train --> to evaluate where the model could improve and TRAIN --> To get actual results
train_prediction = r.predict(df_X).tolist()
test_prediction = r.predict(df_X_Test).tolist()

# Evaluation of the prediction
print('Results are ready!\n')
f = pearsonr(training_scores_y, train_prediction)[0]
g = pearsonr(testing_scores_y, test_prediction)[0]
print('SVR Training Accuracy: ',round(f,3))
print('SVR Testing Accuracy: ', round(g,3))
print('SVR Drop Train-Test: ', round(f-g,3))

/home/jan/anaconda3/envs/IHLT_01/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
r = KNeighborsRegressor(n_neighbors=15)
r.fit(df_X, df_y)
r.score(df_X, df_y)

# do the prediction with train --> to evaluate where the model could improve and TRAIN --> To get actual results
train_prediction = r.predict(df_X).tolist()
test_prediction = r.predict(df_X_Test).tolist()

# Evaluation of the prediction
print('Results are ready!\n')
h = pearsonr(training_scores_y, train_prediction)[0]
i = pearsonr(testing_scores_y, test_prediction)[0]
print('KNN Training Accuracy: ',round(h,3))
print('KNN Testing Accuracy: ', round(i,3))
print('KNN Drop Train-Test: ', round(h-i,3))